## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-09-10-55-00 +0000,wsj,Bank of Japan Governor Sees Little Danger of R...,https://www.wsj.com/articles/bank-of-japan-gov...
1,2025-12-09-10-44-02 +0000,nyt,Another Front in the War in Ukraine: Who Gets ...,https://www.nytimes.com/2025/12/09/world/europ...
2,2025-12-09-10-41-00 +0000,wsj,U.S. Treasury Yields Could Have Room to Fall,https://www.wsj.com/articles/u-s-treasury-yiel...
3,2025-12-09-10-30-00 +0000,wsj,Now Cracker Barrel Diehards Think the Food Isn...,https://www.wsj.com/business/earnings/cracker-...
4,2025-12-09-10-30-00 +0000,wsj,Consumer Loans Are Getting Harder to Tally—and...,https://www.wsj.com/finance/banking/consumer-l...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
293,trump,53
694,warner,18
75,court,16
633,deal,15
157,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
289,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...,168
304,2025-12-08-14-53-40 +0000,latimes,Paramount launches hostile $78-billion bid for...,https://www.latimes.com/business/story/2025-12...,139
311,2025-12-08-14-06-59 +0000,nyt,Trump Looms Over Netflix’s Deal for Warner Bro...,https://www.nytimes.com/2025/12/08/business/de...,125
317,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...,113
102,2025-12-09-01-21-00 +0000,wsj,Behind Paramount’s Relentless Campaign to Woo ...,https://www.wsj.com/business/media/paramount-n...,109


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
289,168,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...
24,79,2025-12-09-09-39-56 +0000,bbc,Ukraine prepares new peace plan as Zelensky ru...,https://www.bbc.com/news/articles/cg7vdd115vjo...
194,60,2025-12-08-21-09-00 +0000,wsj,Supreme Court Appears Ready to Give President ...,https://www.wsj.com/politics/policy/supreme-co...
225,46,2025-12-08-19-24-10 +0000,nyt,"Despite Trump’s Tariffs, China’s Trade Surplus...",https://www.nytimes.com/2025/12/07/business/ch...
105,44,2025-12-09-01-07-00 +0000,wsj,President Trump backed away from a vow to rele...,https://www.wsj.com/politics/national-security...
257,37,2025-12-08-17-05-06 +0000,nypost,"Berkshire Hathaway’s Todd Combs, also CEO of G...",https://nypost.com/2025/12/08/business/berkshi...
302,36,2025-12-08-14-56-20 +0000,nypost,7.2-magnitude earthquake strikes off Japan’s c...,https://nypost.com/2025/12/08/world-news/7-2-m...
179,34,2025-12-08-21-42-03 +0000,nypost,CBS News taps ABC News journalist Matt Gutman ...,https://nypost.com/2025/12/08/media/cbs-news-t...
5,33,2025-12-09-10-30-00 +0000,wsj,Fed Chair Jerome Powell is expected to push fo...,https://www.wsj.com/economy/central-banking/fe...
73,32,2025-12-09-03-33-11 +0000,nypost,Trump will hit the 2026 campaign trail for Rep...,https://nypost.com/2025/12/08/us-news/trump-wi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
